# ELG redshift fitting analysis

In [18]:
import numpy as np
import healpy as hp
import fitsio
import astropy.table
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib import rcParams
import h5py
%pylab inline

# DESI modules
import desispec.io
from desisim.spec_qa import redshifts as dsq_z
from desisim.spec_qa import utils as dsq_u
from desitarget.targetmask import desi_mask
import redrock.templates

Populating the interactive namespace from numpy and matplotlib


In [2]:
froot = '/project/projectdirs/desi/datachallenge/reference_runs/19.2/'
f_truth = froot + 'targets/truth.fits'
f_zcata = froot + 'spectro/redux/mini/zcatalog-mini.fits'

---

### Load data

In [3]:
# truth
truth = Table.read(f_truth, 'TRUTH')
truth_elg = Table.read(f_truth, 'TRUTH_ELG')
truth_bgs = Table.read(f_truth, 'TRUTH_BGS')
truth_lrg = Table.read(f_truth, 'TRUTH_LRG')
truth_qso = Table.read(f_truth, 'TRUTH_QSO')
truth_star = Table.read(f_truth, 'TRUTH_STAR')
truth_wd = Table.read(f_truth, 'TRUTH_WD')

# zcatalog
zcat = Table.read(f_zcata, 'ZCATALOG')

# string wsp strip
truth['TRUESPECTYPE'] = np.char.strip(truth['TRUESPECTYPE'])
truth['TEMPLATETYPE'] = np.char.strip(truth['TEMPLATETYPE'])
zcat['SPECTYPE'] = np.char.strip(zcat['SPECTYPE'])
zcat['SUBTYPE'] = np.char.strip(zcat['SUBTYPE'])

# del EXTNAME for join
del truth.meta['EXTNAME']
del truth_elg.meta['EXTNAME']
del truth_bgs.meta['EXTNAME']
del truth_lrg.meta['EXTNAME']
del truth_qso.meta['EXTNAME']
del truth_star.meta['EXTNAME']
del truth_wd.meta['EXTNAME']
del zcat.meta['EXTNAME']

### Make one table `ztruth`
- Targets:
  1. With TARGETID in both TRUTH and ZCATALOG
  2. DESI_TARGET == ELG
  3. TARGETID >= 0

In [4]:
# select columns
truth_s = truth['TARGETID', 'TRUEZ', 'TRUESPECTYPE', 'TEMPLATETYPE']
zcat_s = zcat['TARGETID', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE', 'SUBTYPE', 'DESI_TARGET', 'HPXPIXEL']
# DESI_TARGET == ELG
idx = (zcat['DESI_TARGET'] & desi_mask.mask('ELG')) != 0
zcat_s = zcat_s[idx]

# join table
ztruth = astropy.table.join(truth_s, zcat_s, keys='TARGETID', join_type='inner')

# append info for TRUTH ELGs
truth_elg_s = truth_elg['TARGETID', 'OIIFLUX', 'OIIDOUBLET']
ztruth = astropy.table.join(ztruth, truth_elg_s, keys='TARGETID', join_type='left')

# check if TARGETID >= 0
assert np.all(ztruth['TARGETID'] >= 0)

print(ztruth.colnames)

['TARGETID', 'TRUEZ', 'TRUESPECTYPE', 'TEMPLATETYPE', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE', 'SUBTYPE', 'DESI_TARGET', 'HPXPIXEL', 'OIIFLUX', 'OIIDOUBLET']


---

### True types check

In [5]:
print('Total number of targets selected w/ DESI_TARGET==ELG: ', len(ztruth), '\n')

print('True Spec Type:')
for st in set(ztruth['TRUESPECTYPE']):
    print('  {:>8s} : {:>6d}'.format(st, np.count_nonzero(ztruth['TRUESPECTYPE']==st)))

print('Template Type:')
for tt in set(ztruth['TEMPLATETYPE']):
    print('  {:>8s} : {:>6d}'.format(tt, np.count_nonzero(ztruth['TEMPLATETYPE']==tt)))

print('Redrock Spec Type:')
for st in set(ztruth['SPECTYPE']):
    print('  {:>8s} : {:>6d}'.format(st, np.count_nonzero(ztruth['SPECTYPE']==st)))

print('Redrock Sub Type:')
for st in set(ztruth['SUBTYPE']):
    print('  {:>8s} : {:>6d}'.format(st, np.count_nonzero(ztruth['SUBTYPE']==st)))

Total number of targets selected w/ DESI_TARGET==ELG:  17204 

True Spec Type:
      STAR :     43
    GALAXY :  17161
Template Type:
      STAR :     43
       BGS :   1576
       ELG :  15585
Redrock Spec Type:
       QSO :      2
    GALAXY :  17202
Redrock Sub Type:
           :  17204


- Some simulated BGS's and STARs are identified as ELGs.
- All simulated ELGs are still identified as ELGs (checked in another notebook).
- Q: How are `DESI_TARGET` types determined?

---

### Get Redrock templates

In [20]:
templates = dict()
for filename in redrock.templates.find_templates():
    t = redrock.templates.Template(filename)
    templates[(t.template_type, t.sub_type)] = t

templates

DEBUG: Read templates from /global/common/software/desi/cori/desiconda/20180709-1.2.6-spec/code/redrock-templates/0.7
DEBUG: Using default redshift range -0.0050-1.6997 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.0500-5.9934 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-CV.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-M.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-WD.fits


{('GALAXY', ''): <redrock.templates.Template at 0x2aaaf23a9a90>,
 ('QSO', ''): <redrock.templates.Template at 0x2aaaf2603668>,
 ('STAR', 'A'): <redrock.templates.Template at 0x2aaaf262dcc0>,
 ('STAR', 'B'): <redrock.templates.Template at 0x2aaaf23a92b0>,
 ('STAR', 'CV'): <redrock.templates.Template at 0x2aaaf25e4cf8>,
 ('STAR', 'F'): <redrock.templates.Template at 0x2aaaf25e4898>,
 ('STAR', 'G'): <redrock.templates.Template at 0x2aaaf2603a58>,
 ('STAR', 'K'): <redrock.templates.Template at 0x2aaaf2603d68>,
 ('STAR', 'M'): <redrock.templates.Template at 0x2aaaf2603588>,
 ('STAR', 'WD'): <redrock.templates.Template at 0x2aaaf2603630>}

---

### Functions

In [22]:
def get_specot(tid):
    '''get the spectra object for one target given the target id'''
    idx = (ztruth['TARGETID'] == tid)
    ipix = ztruth[idx]['HPXPIXEL'][0]
    spec_path = 'spectro/redux/mini/spectra-64/{:d}/{:d}/'.format(ipix//100, ipix)
    specobj = desispec.io.read_spectra(froot + spec_path + 'spectra-64-{:d}.fits'.format(ipix))
    specot = specobj.select(targets=[tid])
    return specot


def get_tempot(tid):
    '''get the best-fit template wave and flux for one target'''
    idx = (ztruth['TARGETID'] == tid)
    spectype = ztruth[idx]['SPECTYPE'][0]
    subtype = ztruth[idx]['SUBTYPE'][0]
    fulltype = (spectype, subtype)
    ncoeff = templates[fulltype].flux.shape[0]
    coeff = fh5['zbest']['coeff'][idx][0][:ncoeff]
    tflux = templates[fulltype].flux.T.dot(coeff)
    z = ztruth[idx]['Z'][0]
    twave = templates[fulltype].wave * (1 + z)
    return twave, tflux

---

### Masks

In [32]:
# goodm fail, miss and lost
objtype_mask, z_mask, survey_mask, dv_mask, zwarn_mask = dsq_z.criteria(ztruth, objtype='ELG')
good_mask = zwarn_mask & dv_mask & objtype_mask & z_mask
fail_mask = zwarn_mask & (~dv_mask) & objtype_mask & z_mask
miss_mask = (~zwarn_mask) & dv_mask & objtype_mask & z_mask
lost_mask = (~zwarn_mask) & (~dv_mask) & objtype_mask & z_mask
tot_mask = objtype_mask & z_mask

ntot = np.count_nonzero(tot_mask)

ngood = np.count_nonzero(good_mask)
nfail = np.count_nonzero(fail_mask)
nmiss = np.count_nonzero(miss_mask)
nlost = np.count_nonzero(lost_mask)

pgood = ngood * 100 / ntot
pfail = nfail * 100 / ntot
pmiss = nmiss * 100 / ntot
plost = nlost * 100 / ntot

print('     ELG     ntarg    good   fail   miss   lost')
print('{:>8s}  {:8d}   {:5d}  {:5d}  {:5d}  {:5d}'.format('number', ntot, ngood, nfail, nmiss, nlost))
print('{:>8s}  {:8.1f}   {:5.1f}  {:5.1f}  {:5.1f}  {:5.1f}'.format('percent', 100.0, pgood, pfail, pmiss, plost))

ztruth_good = ztruth[good_mask]
ztruth_fail = ztruth[fail_mask]
ztruth_miss = ztruth[miss_mask]
ztruth_lost = ztruth[lost_mask]

     ELG     ntarg    good   fail   miss   lost
  number     17204   13918    108   1415   1763
 percent     100.0    80.9    0.6    8.2   10.2


---

## Fail